In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
import random
from collections import defaultdict

In [2]:
pd.set_option('display.max_rows', None)

In [11]:
import sys
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["groupMatch"]
df = pd.DataFrame()
takesCol = db.takes
ls = list(takesCol.find())
for s in ls:
    for c in s["courses"]:
        dt, days, loc, cn = c["startDT"], c["days"], c["loc"], c["courseName"]
        if cn != "EXAMINATION HOUR":
            df = df.append({"ID": s["_id"]["email"], "Student Name": s["studentName"],"Class Name": cn, "Days": days, "Sem": s["_id"]["semester"], "Class Time": dt, "Class Loc": loc}, ignore_index=True)
        
df

,ID,Student Name,Class Name,Days,Sem,Class Time,Class Loc
0,j@nyu.edu,j,ECONOMIC FOUNDATIONS OF FINANC,"TU,TH",2022-01,2022-01-25 15:30:00,2 MetroTechCr Rm 845 845
1,j@nyu.edu,j,CORP FINANCE & FINANCIAL MKTS,WE,2022-01,2022-01-26 17:00:00,"Jacobs Academic Bldg, Rm 473 473"
2,j@nyu.edu,j,GENERAL CHEM FOR ENGINEERS,"TU,TH",2022-01,2022-01-25 14:00:00,"Jacobs Academic Bldg, Rm 474 474"
3,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-01-28 08:00:00,"Dibner Building, Rm LC011 LC011"
4,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-04-29 08:00:00,"Dibner Building, Rm LC011 LC011"
5,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-04-15 08:00:00,"Dibner Building, Rm LC011 LC011"
6,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-04-01 08:00:00,"Dibner Building, Rm LC011 LC011"
7,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-03-11 08:00:00,"Dibner Building, Rm LC011 LC011"
8,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-02-25 08:00:00,"Dibner Building, Rm LC011 LC011"
9,j@nyu.edu,j,INTRO TO EMBEDDED SYSTEM DESIG,FR,2022-01,2022-02-11 08:00:00,"Dibner Building, Rm LC011 LC011"


In [5]:
print(df[df['ID'] == 'a@nyu.edu'])

          ID Student Name                      Class Name   Days      Sem  \
0  a@nyu.edu            a             JAVA AND WEB DESIGN     TU  2022-01   
1  a@nyu.edu            a            Software Engineering  TU,TH  2022-01   
2  a@nyu.edu            a  Advanced Seminar in Urban Stud  TU,TH  2022-01   
3  a@nyu.edu            a     SELECTED TOPICS IN COMP SCI     FR  2022-01   

            Class Time                         Class Loc  
0  2022-01-25 17:00:00  Jacobs Academic Bldg, Rm 475 475  
1  2022-01-25 09:30:00           Rogers Hall, Rm 315 315  
2  2022-01-25 14:00:00         2 MetroTechCtr Rm 804 804  
3  2022-01-28 11:00:00      2 MetroTechCtr Rm 9.007 9007  


In [6]:
dfNames = df['Class Name']
dfLocs = df['Class Loc']
dfTimes = df['Class Time']
dfDays = df['Days']

In [7]:
#TF-IDF the Names First
tfIdfVectorizer = TfidfVectorizer()
XNames = tfIdfVectorizer.fit_transform(dfNames)
print(tfIdfVectorizer.get_feature_names())
print(XNames)

['adv', 'advanced', 'and', 'arabic', 'arts', 'biomolecular', 'chem', 'comp', 'culture', 'databases', 'design', 'eng', 'engineering', 'ethics', 'happiness', 'health', 'ii', 'in', 'intermediate', 'intro', 'introduction', 'java', 'kinetics', 'med', 'medical', 'nutrition', 'of', 'operating', 'reactor', 'sci', 'science', 'selected', 'sem', 'seminar', 'separa', 'software', 'special', 'stud', 'sue', 'systm', 'technology', 'the', 'to', 'topics', 'transport', 'urban', 'web']
  (0, 10)	0.4596391178358206
  (0, 46)	0.5623099460242047
  (0, 2)	0.3954072970490309
  (0, 21)	0.5623099460242047
  (1, 12)	0.7071067811865475
  (1, 35)	0.7071067811865475
  (2, 37)	0.4749038739634264
  (2, 45)	0.4749038739634264
  (2, 17)	0.31283420845252813
  (2, 33)	0.4749038739634264
  (2, 1)	0.4749038739634264
  (3, 29)	0.47401146674419997
  (3, 7)	0.47401146674419997
  (3, 43)	0.4520547160652418
  (3, 31)	0.47401146674419997
  (3, 17)	0.3486917289931182
  (4, 12)	0.7071067811865475
  (4, 35)	0.7071067811865475
  (5, 

C:\Users\swlas\anaconda3\envs\introds\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## .indices on sparse matrix for indexes of all non 0 matches
index with [og row, other row]
Ex.)
row 5 is being checked
want to get the values? 
matrix.indices gives the matching row indexes:
[67 43 28 24 11  5]
then to get the values from the specific row matches:
matrix[5,67] -> 1.0
matrix[5,43] -> .4102

In [8]:
similarities = cosine_similarity(XNames)
similarities_sparse = cosine_similarity(XNames,dense_output=False)
#print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
#print(similarities_sparse)
print(similarities_sparse[14,3])
#print(similarities_sparse[11].sum())
ind = similarities_sparse[14].indices
print(ind)

1.0000000000000002
[39 36 32 29 21  7  2  6 44 40 37 15 14  3]


In [9]:
ans = []
for i in ind:
    print(i)
    print(similarities_sparse[14,i])
    if similarities_sparse[14,i] >= 1:
        ans.append(i)
print(ans)

39
0.09261327585204733
36
0.10908270103350562
32
0.10908270103350562
29
0.09261327585204733
21
0.10908270103350562
7
0.09261327585204733
2
0.10908270103350562
6
0.2914412820038832
44
1.0000000000000002
40
1.0000000000000002
37
1.0000000000000002
15
1.0000000000000002
14
1.0000000000000002
3
1.0000000000000002
[44, 40, 37, 15, 14, 3]


In [10]:
tfIdfVectorizerLoc = TfidfVectorizer()
XLoc = tfIdfVectorizerLoc.fit_transform(dfLocs)
print(tfIdfVectorizerLoc.get_feature_names())
print(XLoc.shape)
print(XLoc)
print(XLoc[0].shape)

['007', '009', '011', '121', '200', '202', '265', '315', '317', '370', '375', '475', '503', '505', '5th', '60', '705', '775b', '803', '804', '812', '9007', '9009', '9011', 'academic', 'ave', 'bldg', 'gcasl', 'hall', 'jacobs', 'jay', 'metrotechctr', 'meyr', 'online', 'rm', 'rogers', 'room', 'street']
(45, 38)
  (0, 11)	0.8002968891738367
  (0, 34)	0.1587918969854355
  (0, 26)	0.3186958769545886
  (0, 24)	0.34113264329479825
  (0, 29)	0.34113264329479825
  (1, 7)	0.8854839765749231
  (1, 28)	0.3064662320516937
  (1, 35)	0.3064662320516937
  (1, 34)	0.16755603377120096
  (2, 19)	0.9502143585793006
  (2, 31)	0.27889517206414133
  (2, 34)	0.13896098642799462
  (3, 21)	0.6414826780520502
  (3, 0)	0.6414826780520502
  (3, 31)	0.3765601314191507
  (3, 34)	0.18762306612975746
  (4, 7)	0.8854839765749231
  (4, 28)	0.3064662320516937
  (4, 35)	0.3064662320516937
  (4, 34)	0.16755603377120096
  (5, 33)	1.0
  (6, 12)	0.9391399067148278
  (6, 28)	0.2265734174889812
  (6, 35)	0.2265734174889812
  (6,

In [11]:
similarities = cosine_similarity(XLoc)
similarities_sparse = cosine_similarity(XLoc,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
print(similarities_sparse[1])

pairwise sparse output:
   (0, 33)	0.0690409873141891
  (0, 9)	0.0690409873141891
  (0, 44)	0.023710367716862385
  (0, 43)	0.02303643062720084
  (0, 42)	0.026606540453884702
  (0, 40)	0.023710367716862385
  (0, 39)	0.023710367716862385
  (0, 37)	0.029793022588968005
  (0, 36)	0.022065878641868632
  (0, 35)	0.026606540453884702
  (0, 32)	0.021519576891512647
  (0, 31)	0.02303643062720084
  (0, 29)	0.023710367716862385
  (0, 28)	0.02303643062720084
  (0, 27)	0.02303643062720084
  (0, 25)	0.32316131126698444
  (0, 24)	0.02303643062720084
  (0, 23)	0.029793022588968005
  (0, 22)	0.32316131126698444
  (0, 21)	0.021519576891512647
  (0, 20)	0.02303643062720084
  (0, 18)	0.32316131126698444
  (0, 17)	0.029793022588968005
  (0, 16)	0.32316131126698444
  (0, 15)	0.023710367716862385
  :	:
  (44, 20)	0.021661912824725845
  (44, 19)	0.023710367716862385
  (44, 18)	0.02131221996760164
  (44, 16)	0.02131221996760164
  (44, 13)	0.025019004428535645
  (44, 11)	0.018496789344769008
  (44, 10)	0.025019

In [12]:
tfIdfVectorizerTimes = TfidfVectorizer()
XTimes = tfIdfVectorizerTimes.fit_transform(dfTimes)
print(tfIdfVectorizerTimes.get_feature_names())
print(XTimes.shape)
print(XTimes)

['00', '01', '08', '09', '11', '12', '14', '16', '17', '18', '2022', '24', '25', '26', '27', '28', '30', '55']
(45, 18)
  (0, 0)	0.46370655676507977
  (0, 8)	0.746382881558645
  (0, 12)	0.34695379798443576
  (0, 1)	0.23185327838253988
  (0, 10)	0.23185327838253988
  (1, 16)	0.5895077791213494
  (1, 3)	0.6583150954895121
  (1, 0)	0.20449626708911262
  (1, 12)	0.3060157571856455
  (1, 1)	0.20449626708911262
  (1, 10)	0.20449626708911262
  (2, 6)	0.5942052722932764
  (2, 0)	0.560414731123948
  (2, 12)	0.41931263764377863
  (2, 1)	0.280207365561974
  (2, 10)	0.280207365561974
  (3, 4)	0.5434753351940984
  (3, 15)	0.6533783198258503
  (3, 0)	0.4302955040567753
  (3, 1)	0.21514775202838765
  (3, 10)	0.21514775202838765
  (4, 16)	0.5895077791213494
  (4, 3)	0.6583150954895121
  (4, 0)	0.20449626708911262
  (4, 12)	0.3060157571856455
  :	:
  (40, 6)	0.5942052722932764
  (40, 0)	0.560414731123948
  (40, 12)	0.41931263764377863
  (40, 1)	0.280207365561974
  (40, 10)	0.280207365561974
  (41, 0)	0

In [13]:
similarities = cosine_similarity(XTimes)
similarities_sparse = cosine_similarity(XTimes,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
print(similarities_sparse[1])

pairwise sparse output:
   (0, 44)	0.5352840901682514
  (0, 43)	0.44291259411621386
  (0, 42)	0.49960512738865565
  (0, 41)	1.0000000000000002
  (0, 40)	0.5352840901682514
  (0, 39)	0.3018863899800503
  (0, 38)	1.0000000000000002
  (0, 37)	0.29929626986649255
  (0, 36)	0.5352840901682514
  (0, 35)	0.29582584896505165
  (0, 34)	1.0000000000000002
  (0, 33)	0.3166110341058325
  (0, 32)	0.40587658629156026
  (0, 31)	0.5352840901682514
  (0, 30)	0.2942782600773259
  (0, 29)	0.3018863899800503
  (0, 28)	0.44291259411621386
  (0, 27)	0.5352840901682514
  (0, 26)	0.2942782600773259
  (0, 25)	0.3132678986668111
  (0, 24)	0.44291259411621386
  (0, 23)	0.18000512359193868
  (0, 22)	0.49960512738865565
  (0, 21)	0.40587658629156026
  (0, 20)	0.5352840901682514
  :	:
  (44, 11)	0.3483920151280102
  (44, 10)	0.21754603523929755
  (44, 9)	0.16189834459506586
  (44, 8)	0.33237645681298067
  (44, 7)	0.36484621060978223
  (44, 6)	0.6038001167956403
  (44, 5)	0.4905240494395785
  (44, 4)	0.3575217153792

In [14]:
tfIdfVectorizerDays = TfidfVectorizer()
XDays = tfIdfVectorizerDays.fit_transform(dfDays)
print(tfIdfVectorizerDays.get_feature_names())
print(XDays)

['fr', 'mo', 'th', 'tu', 'we']
  (0, 3)	1.0
  (1, 2)	0.7653224202383335
  (1, 3)	0.6436471029069731
  (2, 2)	0.7653224202383335
  (2, 3)	0.6436471029069731
  (3, 0)	1.0
  (4, 2)	0.7653224202383335
  (4, 3)	0.6436471029069731
  (5, 2)	0.7653224202383335
  (5, 3)	0.6436471029069731
  (6, 2)	0.7653224202383335
  (6, 3)	0.6436471029069731
  (7, 4)	0.6785432194748259
  (7, 1)	0.7345604803586552
  (8, 0)	1.0
  (9, 4)	1.0
  (10, 4)	0.6785432194748259
  (10, 1)	0.7345604803586552
  (11, 4)	0.6785432194748259
  (11, 1)	0.7345604803586552
  (12, 2)	0.7653224202383335
  (12, 3)	0.6436471029069731
  (13, 2)	0.7653224202383335
  (13, 3)	0.6436471029069731
  (14, 4)	1.0
  :	:
  (29, 1)	0.7345604803586552
  (30, 2)	1.0
  (31, 3)	1.0
  (32, 2)	0.7653224202383335
  (32, 3)	0.6436471029069731
  (33, 4)	0.573128811129294
  (33, 1)	0.6204435660507523
  (33, 2)	0.40969643303759945
  (33, 3)	0.3445605606508317
  (34, 3)	1.0
  (35, 2)	0.7653224202383335
  (35, 3)	0.6436471029069731
  (36, 2)	0.76532242023833

In [15]:
similarities = cosine_similarity(XDays)
similarities_sparse = cosine_similarity(XDays,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
print(similarities_sparse[1])

pairwise sparse output:
   (0, 44)	1.0
  (0, 43)	0.6436471029069731
  (0, 42)	0.6436471029069731
  (0, 41)	1.0
  (0, 40)	1.0
  (0, 38)	1.0
  (0, 36)	0.6436471029069731
  (0, 35)	0.6436471029069731
  (0, 34)	1.0
  (0, 33)	0.34456056065083174
  (0, 32)	0.6436471029069731
  (0, 31)	1.0
  (0, 28)	0.6436471029069731
  (0, 27)	1.0
  (0, 24)	0.6436471029069731
  (0, 22)	0.6436471029069731
  (0, 21)	0.6436471029069731
  (0, 20)	1.0
  (0, 16)	0.6436471029069731
  (0, 15)	1.0
  (0, 13)	0.6436471029069731
  (0, 12)	0.6436471029069731
  (0, 6)	0.6436471029069731
  (0, 5)	0.6436471029069731
  (0, 4)	0.6436471029069731
  :	:
  (44, 41)	1.0
  (44, 40)	1.0
  (44, 38)	1.0
  (44, 36)	0.6436471029069731
  (44, 35)	0.6436471029069731
  (44, 34)	1.0
  (44, 33)	0.34456056065083174
  (44, 32)	0.6436471029069731
  (44, 31)	1.0
  (44, 28)	0.6436471029069731
  (44, 27)	1.0
  (44, 24)	0.6436471029069731
  (44, 22)	0.6436471029069731
  (44, 21)	0.6436471029069731
  (44, 20)	1.0
  (44, 16)	0.6436471029069731
  (44

In [16]:
def getOnes(sim,user):
    ind = sim[user].indices
    ans =[]
    for i in ind:
        if sim[user,i] >= 1.0:
            ans.append(i)
    return ans

In [17]:
def getSimMatrix(df):
    tfIdfVectorizer = TfidfVectorizer()
    X = tfIdfVectorizerLoc.fit_transform(df)
    similarities = cosine_similarity(X)
    similarities_sparse = cosine_similarity(X,dense_output=False)
    #print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
    #print(similarities_sparse[1])
    return similarities_sparse

In [18]:
students = df.ID.unique()
students

array(['a@nyu.edu', 'b@nyu.edu', 'c@nyu.edu', 'd@nyu.edu', 'e@nyu.edu',
       'f@nyu.edu', 'g@nyu.edu', 'h@nyu.edu', 'i@nyu.edu', 'j@nyu.edu'],
      dtype=object)

In [19]:
def randCentroids(students,k):
    centroidID = random.sample(list(students),k)
    return centroidID

In [20]:
def getMatches(df,u,Name,Loc,Time,Day):
    matches = []
    nameMatches = getOnes(Name,u)
    locMatches = getOnes(Loc,u)
    timeMatches = getOnes(Time,u)
    dayMatches = getOnes(Day,u)
    for k in range(len(nameMatches)):
        if nameMatches[k] in locMatches and nameMatches[k] in timeMatches and nameMatches[k] in dayMatches:
            matches.append(nameMatches[k])
    return matches

In [21]:
def getOnes(sim,course):
    ind = sim[course].indices
    ans =[]
    for i in ind:
        if sim[course,i] >= 1.0:
            ans.append(i)
    return ans

In [22]:
def findEqivRow(df,df2,ind):
    index = df.index
    condition = df2[(df2["ID"].isin([df["ID"].iloc[ind]])) & (df2['Class Name'] == df.iloc[ind]["Class Name"]) & (df2['Class Loc'] == df.iloc[ind]["Class Loc"]) & (df2['Class Time'] == df.iloc[ind]["Class Time"]) & (df2['Days'] == df.iloc[ind]["Days"])].ID
    otherInd = condition.index.tolist()
    return otherInd

In [23]:
def findMatch(df,course,studentID,nameSim,locSim,timeSim,daySim):
    matches = getMatches(df,course,nameSim,locSim,timeSim,daySim)
    for i in matches:
        if df.iloc[i].ID == studentID:
            return i

In [24]:
def matchAll(df,k,threshold,count):
    def helper(df,k,threshold,count):
        nonlocal MATCHES
        tempDF = df.copy(deep=True) #dataframe to update for recursive call
        students = df.ID.unique() #list of user id
        centroidID = randCentroids(students,k) #list of user id of all centroids
        centroidID = sorted(centroidID) #sort the centroids for the first pass
        noCents = [x for x in students if x not in centroidID] #list of noncentroid students
        g = [] # list of lists of IDs that match
        for i in range(k):
            g.append([])
        totalMatches = []
        groups = {}

        for i in range(k):
            groups[(centroidID[i], str(df.loc[df["ID"] == centroidID[i]]["Student Name"].iloc[0]))] = [[], defaultdict(set)]

        #indexes of centroids for matches
        index = df.index
        condition = df['ID'].isin(centroidID)
        centroidInd = index[condition].tolist()
        counts = dict((el,0) for el in centroidID)

        #If there are meaningful matches within the centroids
        if threshold >= 2:


            for centroid in centroidID:
                totalMatches = [] #list of all matches for each student

                counts = dict((el,0) for el in centroidID) #Counts dictionary keeps track of matches

                #Get the indexes of the classes for the current student
                index = df.index
                condition = df['ID'].isin([centroid])
                courses = index[condition].tolist()

                #Run the similarity matrix for all their features
                nameSim = getSimMatrix(df['Class Name'])
                locSim = getSimMatrix(df['Class Loc'])
                timeSim = getSimMatrix(df['Class Time'])
                daySim = getSimMatrix(df['Days'])

                #For each class each that the student has
                for course in courses:

                    matches = getMatches(df,course,nameSim,locSim,timeSim,daySim) #Class matches with other students

                    #check to see if the match is a centroid otherwise discard it
                    for match in matches:
                        if match in centroidInd and df.iloc[match].ID != centroid:

                            cID = df.iloc[match].ID
                            totalMatches.append(match) #append to our totalMatches
                            counts[cID] +=1  #Increment the counts dictionary for the centroid

                bestCluster = max(counts, key = counts.get) #id of the centroid with best match

                #checking which centroid to group to
                for i in range(k):

                    if bestCluster == centroidID[i] and len(g[i]) < 10 and counts[bestCluster] >= threshold: #conditions are met
                        stu = str(df.loc[df["ID"] == centroid]["Student Name"].iloc[0])
                        g[i].append([centroid, stu])
                        name = str(df.loc[df["ID"] == centroidID[i]]["Student Name"].iloc[0])
                        groups[(centroidID[i], name)][0] = g[i]
                        for t in totalMatches:
                            groups[(centroidID[i], name)][1][tuple(df.iloc[t].drop("ID").drop("Student Name"))].add((centroidID[i], name))
                            groups[(centroidID[i], name)][1][tuple(df.iloc[t].drop("ID").drop("Student Name"))].add((centroid, stu))

                        #Drop the centroid thats matched with another
                        groups.pop(centroid,None)
                        noCents = [x for x in students if x not in centroidID]

                        #Update centroidID
                        newChoice = randCentroids(noCents,1)
                        centroidID.remove(centroid)
                        centroidID.append(newChoice[0])
                        index = df.index
                        condition = df['ID'].isin(centroidID)
                        centroidInd = index[condition].tolist()
                        groups[centroidID[-1]] = []

                        #drop classes that matched
                        for ind in totalMatches:
                            if df.iloc[ind].ID == bestCluster:
                                #Find where it has a match with the centroid
                                cMatch = findMatch(df,ind,centroid,nameSim,locSim,timeSim,daySim)
                                #drop the appropriate index
                                tempDF.drop([cMatch],inplace = True)
                                tempDF = tempDF.reset_index(drop=True)

                            elif df.iloc[ind].ID == centroid:

                                tempDF.drop([ind],inplace=True)
                                tempDF = tempDF.reset_index(drop=True)

        #Create a list with the student IDS for all Non Centroid students
        notCentroids = [x for x in students if x not in centroidID]

        #Create a list with the indices for all centroid's classes
        index = df.index
        condition = df['ID'].isin(centroidID)
        centroidInd = index[condition].tolist()


        for student in notCentroids:
            #Create a list with the indices for all centroid's classes
            totalMatches = []
            #Counts dictionary will see how many similar classes a student has with a centroid
            counts = dict((el,0) for el in centroidID)

            #indexes for students classes
            index = df.index
            condition = df['ID'].isin([student])
            courses = index[condition].tolist()

            #get feature similarities
            nameSim = getSimMatrix(df['Class Name'])
            locSim = getSimMatrix(df['Class Loc'])
            timeSim = getSimMatrix(df['Class Time'])
            daySim = getSimMatrix(df['Days'])

            #for each course the student has
            for course in courses:


                matches = getMatches(df,course,nameSim,locSim,timeSim,daySim) #list of matches

                #check to see if the matches are centroids otherwise discard them
                for match in matches:
                    if match in centroidInd and df.iloc[match].ID != student:
                        cID = df.iloc[match].ID
                        totalMatches.append(match)
                        counts[cID] +=1

            bestCluster = max(counts,key=counts.get) #centroid the student has the most matches with

            #match it to the proper dictionary key
            for i in range(k):
                if bestCluster == centroidID[i] and len(g[i]) < 10 and counts[bestCluster] >= threshold:

                    #append to the appropriate group
                    stu = str(df.loc[df["ID"] == student]["Student Name"].iloc[0])
                    g[i].append([student, stu])
                    name = str(df.loc[df["ID"] == centroidID[i]]["Student Name"].iloc[0])
                    groups[(centroidID[i], name)][0] = g[i]
                    for t in totalMatches:
                        groups[(centroidID[i], name)][1][tuple(df.iloc[t].drop("ID").drop("Student Name"))].add((centroidID[i], name))
                        groups[(centroidID[i], name)][1][tuple(df.iloc[t].drop("ID").drop("Student Name"))].add((student, stu))

                    #drop the class so it doesn't get matched again
                    for ind in totalMatches:

                        if df.iloc[ind].ID == bestCluster:
                            cMatch = findMatch(df,ind,student,nameSim,locSim,timeSim,daySim) #the index the students class
                            otherInd = findEqivRow(df,tempDF,cMatch) #find the eqivalent index for the smaller dataframe
                            tempDF.drop(otherInd,inplace=True) #drop the class
                            tempDF = tempDF.reset_index(drop=True)
                        elif df.iloc[ind].ID == student:
                            tempDF.drop([ind],inplace=True) #drop if the index is correct
                            tempDF = tempDF.reset_index(drop=True)
                        else:
                            tempDF = tempDF.reset_index(drop=True) #reset index
        # if thredhold > 1 and there are at least 5 students left to be centroids, lower threshold and recurse
        if threshold > 1 and len(tempDF.ID.unique()) >= 5: 
            if count >= 5:
                threshold -=1
                helper(tempDF,5,threshold,0)
            else:
                count +=1
                helper(tempDF,5,threshold,count)
        # if threshold is 1 but there are still students to match, recurse without lowering threshold
        elif threshold == 1 and len(tempDF.ID.unique()) >= 5 :
            if count >= 5:
                #print("done matching\n")
                pass
            else:
                count +=1
                helper(tempDF,5,threshold,count)

        new_dict = dict((k,v) for k,v in groups.items() if v[0])
        match = {}
        
        if new_dict:
            for centroid in new_dict:
                matches, courses = new_dict[centroid]
                ml = list(matches)
                ml.extend([list(centroid)])
                MATCHES.append({"_id": {}, "matchingCourses": [], "students": []})
                MATCHES[-1]["_id"]["groupName"] = " ".join(em for em, sn in ml)
                print(centroid, matches)
                for c, students in courses.items():
                    cn, day, sem, sdt, loc = c
                    MATCHES[-1]["_id"]["semester"] = sem
                    keys = ["courseName", "day", "startDT", "loc", "studentsTaking"]
                    MATCHES[-1]["matchingCourses"].append(dict(zip(keys, [cn, day, sdt, loc, []])))
                    for em, sn in students:
                        MATCHES[-1]["matchingCourses"][-1]["studentsTaking"].append({"studentName":sn})
                    print(c, students)
                print()
                keys = ["email", "studentName"]
                
                for em, sn in ml:
                    MATCHES[-1]["students"].append(dict(zip(keys,[em,sn])))
    MATCHES = []
    
    helper(df,5,1,1)
    return MATCHES

In [25]:
allGroups = matchAll(df,5,1,1)
allGroups

('f@nyu.edu', 'f') [['g@nyu.edu', 'g']]
('Ethics and Technology', 'TU,TH', '2022-01', '2022-01-25 08:00:00', 'Rogers Hall, Rm 317 317') {('g@nyu.edu', 'g'), ('f@nyu.edu', 'f')}

('e@nyu.edu', 'e') [['g@nyu.edu', 'g']]
('Ethics and Technology', 'TU,TH', '2022-01', '2022-01-25 08:00:00', 'Rogers Hall, Rm 317 317') {('e@nyu.edu', 'e'), ('g@nyu.edu', 'g')}
('TRANSPORT II', 'TH', '2022-01', '2022-01-27 14:00:00', 'Jacobs Academic Bldg, Rm 475 475') {('e@nyu.edu', 'e'), ('g@nyu.edu', 'g')}
('CHEM & BIOMOLECULAR ENG SEPARA', 'TU', '2022-01', '2022-01-25 14:00:00', 'Rogers Hall, Rm 705 705') {('e@nyu.edu', 'e'), ('g@nyu.edu', 'g')}
('Advanced Seminar in Urban Stud', 'TU,TH', '2022-01', '2022-01-25 18:00:00', 'Rogers Hall, Rm 200 200') {('e@nyu.edu', 'e'), ('g@nyu.edu', 'g')}

('h@nyu.edu', 'h') [['i@nyu.edu', 'i'], ['j@nyu.edu', 'j']]
('JAVA AND WEB DESIGN', 'TU', '2022-01', '2022-01-25 17:00:00', 'Jacobs Academic Bldg, Rm 475 475') {('j@nyu.edu', 'j'), ('h@nyu.edu', 'h'), ('i@nyu.edu', 'i')}


[{'_id': {'groupName': 'g@nyu.edu f@nyu.edu', 'semester': '2022-01'},
  'matchingCourses': [{'courseName': 'Ethics and Technology',
    'day': 'TU,TH',
    'startDT': '2022-01-25 08:00:00',
    'loc': 'Rogers Hall, Rm 317 317',
    'studentsTaking': [{'studentName': 'g'}, {'studentName': 'f'}]}],
  'students': [{'email': 'g@nyu.edu', 'studentName': 'g'},
   {'email': 'f@nyu.edu', 'studentName': 'f'}]},
 {'_id': {'groupName': 'g@nyu.edu e@nyu.edu', 'semester': '2022-01'},
  'matchingCourses': [{'courseName': 'Ethics and Technology',
    'day': 'TU,TH',
    'startDT': '2022-01-25 08:00:00',
    'loc': 'Rogers Hall, Rm 317 317',
    'studentsTaking': [{'studentName': 'e'}, {'studentName': 'g'}]},
   {'courseName': 'TRANSPORT II',
    'day': 'TH',
    'startDT': '2022-01-27 14:00:00',
    'loc': 'Jacobs Academic Bldg, Rm 475 475',
    'studentsTaking': [{'studentName': 'e'}, {'studentName': 'g'}]},
   {'courseName': 'CHEM & BIOMOLECULAR ENG SEPARA',
    'day': 'TU',
    'startDT': '2022-01

In [26]:
groupsCol = db["groups"]
groupsDf = pd.DataFrame(list(groupsCol.find()))
for g in allGroups:
    q = { "groupName": g["_id"]["groupName"] }
    if db.mycollection.count_documents(q) == 0:
        try: 
            insIds = groupsCol.insert_one(g)
            print(insIds.inserted_id)
        except:
            print(g["_id"]["groupName"], " exists already")
    else:
        print(g["_id"]["groupName"], " exists already")

{'groupName': 'g@nyu.edu f@nyu.edu', 'semester': '2022-01'}
{'groupName': 'g@nyu.edu e@nyu.edu', 'semester': '2022-01'}
{'groupName': 'i@nyu.edu j@nyu.edu h@nyu.edu', 'semester': '2022-01'}
{'groupName': 'j@nyu.edu f@nyu.edu', 'semester': '2022-01'}
{'groupName': 'b@nyu.edu h@nyu.edu', 'semester': '2022-01'}
{'groupName': 'i@nyu.edu j@nyu.edu d@nyu.edu', 'semester': '2022-01'}
{'groupName': 'f@nyu.edu e@nyu.edu', 'semester': '2022-01'}
{'groupName': 'b@nyu.edu g@nyu.edu', 'semester': '2022-01'}
{'groupName': 'a@nyu.edu h@nyu.edu', 'semester': '2022-01'}
